In [7]:
# ✅ Rutas de los archivos y modelo
ruta_test_csv = r'C:\Users\mikel\OneDrive\Documentos\TB-DS-BIO-23.09.24\REPOSITORIOS\Mikel\clip_count\test.csv'
ruta_test_img = r'C:\Users\mikel\OneDrive\Documentos\TB-DS-BIO-23.09.24\REPOSITORIOS\Mikel\clip_count\test'
ruta_modelo = r'C:\Users\mikel\OneDrive\Documentos\TB-DS-BIO-23.09.24\REPOSITORIOS\Mikel\dsft2409_mikeltelo\PROJECTS\kaggle_clip_counting\modelo_final\modelo_clip_count_p.h5'
ruta_resultados = r'C:\Users\mikel\OneDrive\Documentos\TB-DS-BIO-23.09.24\REPOSITORIOS\Mikel\dsft2409_mikeltelo\PROJECTS\kaggle_clip_counting\submission_p_final.csv'

In [8]:
from tensorflow.keras.models import load_model

# ✅ Cargar el modelo entrenado
modelo = load_model(ruta_modelo)
print(f"✅ Modelo cargado correctamente desde '{ruta_modelo}'")


✅ Modelo cargado correctamente desde 'C:\Users\mikel\OneDrive\Documentos\TB-DS-BIO-23.09.24\REPOSITORIOS\Mikel\dsft2409_mikeltelo\PROJECTS\kaggle_clip_counting\modelo_final\modelo_clip_count_p.h5'


In [9]:
import tensorflow as tf
import pandas as pd
import numpy as np

# ✅ Cargar datos de prueba
df_test = pd.read_csv(ruta_test_csv)

# ✅ Función de preprocesamiento para imágenes de prueba
IMG_SIZE = (128, 128)
BATCH_SIZE = 32
AUTOTUNE = tf.data.AUTOTUNE

def cargar_datos_test(df, ruta_img):
    ids = df['id'].values
    
    def preprocesar_imagen(id):
        img_path = tf.strings.join([ruta_img, "/clips-", tf.strings.as_string(id), ".png"])
        img = tf.io.read_file(img_path)
        img = tf.image.decode_png(img, channels=1)
        img = tf.image.resize(img, IMG_SIZE)
        img = img / 255.0
        return img
    
    dataset = tf.data.Dataset.from_tensor_slices(ids)
    dataset = dataset.map(lambda id: preprocesar_imagen(id), num_parallel_calls=AUTOTUNE)
    dataset = dataset.batch(BATCH_SIZE).prefetch(AUTOTUNE)
    return dataset

# ✅ Crear Dataset de Prueba
dataset_test = cargar_datos_test(df_test, ruta_test_img)


In [10]:
# ✅ Generar predicciones
predicciones = modelo.predict(dataset_test).flatten()
df_test['clip_count'] = np.round(predicciones).astype(int)
print("✅ Predicciones generadas correctamente.")

157/157 ━━━━━━━━━━━━━━━━━━━━ 10s 62ms/step
✅ Predicciones generadas correctamente.


In [11]:
# ✅ Guardar las predicciones en un archivo CSV
df_test[['id', 'clip_count']].to_csv(ruta_resultados, index=False)
print(f"✅ Archivo de predicciones guardado en '{ruta_resultados}'")

✅ Archivo de predicciones guardado en 'C:\Users\mikel\OneDrive\Documentos\TB-DS-BIO-23.09.24\REPOSITORIOS\Mikel\dsft2409_mikeltelo\PROJECTS\kaggle_clip_counting\submission_p_final.csv'
